In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Load and Explore the Data

We'll be loading in some data about global energy consumption, collected over a few years. The below cell downloads and unzips this data, giving you one text file of data, `household_power_consumption.txt`.

In [ ]:
! wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/March/5c88a3f1_household-electric-power-consumption/household-electric-power-consumption.zip
! unzip household-electric-power-consumption

### Read in the `.txt` File

The next cell displays the first few lines in the text file, so we can see how it is formatted.

In [ ]:
# display first ten lines of text data
n_lines = 10

with open('household_power_consumption.txt') as file:
    head = [next(file) for line in range(n_lines)]
    
display(head)

## Pre-Process the Data

The 'household_power_consumption.txt' file has the following attributes:
   * Each data point has a date and time (hour:minute:second) of recording
   * The various data features are separated by semicolons (;)
   * Some values are 'nan' or '?', and we'll treat these both as `NaN` values

### Managing `NaN` values

This DataFrame does include some data points that have missing values. So far, we've mainly been dropping these values, but there are other ways to handle `NaN` values, as well. One technique is to just fill the missing column values with the **mean** value from that column; this way the added value is likely to be realistic.

I've provided some helper functions in `txt_preprocessing.py` that will help to load in the original text file as a DataFrame *and* fill in any `NaN` values, per column, with the mean feature value. This technique will be fine for long-term forecasting; if I wanted to do an hourly analysis and prediction, I'd consider dropping the `NaN` values or taking an average over a small, sliding window rather than an entire column of data.

**Below, I'm reading the file in as a DataFrame and filling `NaN` values with feature-level averages.**

In [ ]:
import txt_preprocessing as pprocess

# create df from text file
initialdf = pprocess.create_df('household_power_consumption.txt', sep=';')

# fill NaN column values with *average* column value
helperdf = pprocess.fill_nan_with_mean(initialdf)

# print some stats about the data
print('Data shape: ', helperdf.shape)
df.head()

## Global Active Power 

In this example, we'll want to predict the global active power, which is the household minute-averaged active power (kilowatt), measured across the globe. So, below, I am getting just that column of data and displaying the resultant plot.

In [ ]:
# Select Global active power data
powerdf = helperdf['Global_active_power'].copy()
print(powerdf.shape)

In [ ]:
# display the data 
plt.figure(figsize=(12,6))
# all data points
powerdf.plot(title='Global active power', color='black') 
plt.show()

Since the data is recorded each minute, the above plot contains *a lot* of values. So, I'm also showing just a slice of data, below.

In [ ]:
# can plot a slice of hourly data
timeslice = 1440 # 1440 mins = 1 day

plt.figure(figsize=(12,6))
powerdf[0:timeslice].plot(title='Global active power, over one day', color='black') 
plt.show()

### Hourly vs Daily

There is a lot of data, collected every minute, and so I could go one of two ways with my analysis:
1. Create many, short time series, say a week or so long, in which I record energy consumption every hour, and try to predict the energy consumption over the following hours or days.
2. Create fewer, long time series with data recorded daily that I could use to predict usage in the following weeks or months.

Both tasks are interesting! It depends on whether you want to predict time patterns over a day/week or over a longer time period, like a month. With the amount of data I have, I think it would be interesting to see longer, *recurring* trends that happen over several months or over a year. So, I will resample the 'Global active power' values, recording **daily** data points as averages over 24-hr periods.

> I can resample according to a specified frequency, by utilizing pandas [time series tools](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html), which allow me to sample at points like every hour ('H') or day ('D'), etc.



In [ ]:
# resample over day (D)
frequency = 'D'
# calculate the mean active power for a day
meanpowerdf=powerdf.resample(frequency).mean #frequency is a column in this case 'daily'

# display the mean values
plt.figure(figsize=(15,8))
meanpowerdf.plot(title='Global active power, mean per day', color='black') 
plt.tight_layout()
plt.show()

In this plot, we can see that there are some interesting trends that occur over each year. It seems that there are spikes of energy consumption around the end/beginning of each year, which correspond with heat and light usage being higher in winter months. We also see a dip in usage around August, when global temperatures are typically higher.

The data is still not very smooth, but it shows noticeable trends, and so, makes for a good use case for machine learning models that may be able to recognize these patterns.

---
## Create Time Series 

My goal will be to take full years of data, from 2007-2009, and see if I can use it to accurately predict the average Global active power usage for the next several months in 2010!

Next, let's make one time series for each complete year of data. This is just a design decision, and I am deciding to use full years of data, starting in January of 2017 because there are not that many data points in 2006 and this split will make it easier to handle leap years; I could have also decided to construct time series starting at the first collected data point, just by changing `t_start` and `t_end` in the function below.

The function `make_time_series` will create pandas `Series` for each of the passed in list of years `['2007', '2008', '2009']`.
* All of the time series will start at the same time point `t_start` (or t0). 
    * When preparing data, it's important to use a consistent start point for each time series; DeepAR uses this time-point as a frame of reference, which enables it to learn recurrent patterns e.g. that weekdays behave differently from weekends or that Summer is different than Winter.
    * You can change the start and end indices to define any time series you create.
* We should account for leap years, like 2008, in the creation of time series.
* Generally, we create `Series` by getting the relevant global consumption data (from the DataFrame) and date indices.

```
# get global consumption data
data = mean_power_df[start_idx:end_idx]

# create time series for the year
index = pd.DatetimeIndex(start=t_start, end=t_end, freq='D')
time_series.append(pd.Series(data=data, index=index))
```

In [ ]:
def make_time_series(mmeanpowerdf, years, frequency='D', startindex=16):

    timeseries = []
    leapyear = '2008'

    for y in range(len(years)):

        yr = years[y]
        if(year == leapyear):
            endindex = startindex+366
        else:
            endindex = startindex+365

        starttime = yr  + '-01-01' # Jan 1st 
        endtime = yr + '-12-31' # Dec 31st 

        globalconsumption = meanpowerdf[startindex:endindex]

        pd.DatetimeIndex(start=startindex, end=endindex, freq=frequency)
        timeseries.append(pd.Series(data=timeseries, index=idx))
        
        startindex=endindex
        
    return timeseries
    

## Test the results

Below, let's construct one time series for each complete year of data, and display the results.

In [ ]:
years=['2007', '2008', '2009']
frequency='D'
timeseries=make_time_series(meanpowerdf, years,freq=frequency) # function defined dabove

In [ ]:
# display first time series
tsi = 0

plt.figure(figsize=(12,6))
timeseries[tsi].plot(label='test', lw=3, color='black')
timeseries[tsi].plot(label='train', ls=':', color='black')
plt.legend()
plt.show()

---
# Splitting in Time

We'll evaluate our model on a test set of data. For machine learning tasks like classification, we typically create train/test data by randomly splitting examples into different sets. For forecasting it's important to do this train/test split in **time** rather than by individual data points. 
> In general, we can create training data by taking each of our *complete* time series and leaving off the last `prediction_length` data points to create *training* time series. 

### EXERCISE: Create training time series

Complete the `create_training_series` function, which should take in our list of complete time series data and return a list of truncated, training time series.

* In this example, we want to predict about a month's worth of data, and we'll set `prediction_length` to 30 (days).
* To create a training set of data, we'll leave out the last 30 points of *each* of the time series we just generated, so we'll use only the first part as training data. 
* The **test set contains the complete range** of each time series.


In [ ]:
# create truncated, training time series
def create_training_series(timeserieslist, nrpointstopredict):
    trainserieslist=[]
    for s in timeserieslist:
        trainserieslist.append(s[:-nrpointstopredict])
        
        return: trainserieslist
       

    pass

In [ ]:
predictionlength = 30 # 30 days
trainseries = create_training_series(timeserieslist, predictionlength)

### Training and Test Series

We can visualize what these series look like, by plotting the train/test series on the same axis. We should see that the test series contains all of our data in a year, and a training series contains all but the last `prediction_length` points.

In [ ]:
# display train/test time series
tsindex = 0

plt.figure(figsize=(15,8))
# test data is the whole time series
timeseries[tsindex].plot(label='test', lw=3, color=black)
# train data is all but the last prediction pts
trainseries[tsindex].plot(label='train', ls=':', lw=3, color='black')

plt.legend()
plt.show()

## Convert to JSON 

According to the [DeepAR documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html), DeepAR expects to see input training data in a JSON format, with the following fields:

* **start**: A string that defines the starting date of the time series, with the format 'YYYY-MM-DD HH:MM:SS'.
* **target**: An array of numerical values that represent the time series.
* **cat** (optional): A numerical array of categorical features that can be used to encode the groups that the record belongs to. This is useful for finding models per class of item, such as in retail sales, where you might have {'shoes', 'jackets', 'pants'} encoded as categories {0, 1, 2}.

The input data should be formatted with one time series per line in a JSON file. Each line looks a bit like a dictionary, for example:
```
{"start":'2007-01-01 00:00:00', "target": [2.54, 6.3, ...], "cat": [1]}
{"start": "2012-01-30 00:00:00", "target": [1.0, -5.0, ...], "cat": [0]} 
...
```
In the above example, each time series has one, associated categorical feature and one time series feature.

### EXERCISE: Formatting Energy Consumption Data

For our data:
* The starting date, "start," will be the index of the first row in a time series, Jan. 1st of that year.
* The "target" will be all of the energy consumption values that our time series holds.
* We will not use the optional "cat" field.

Complete the following utility function, which should convert `pandas.Series` objects into the appropriate JSON strings that DeepAR can consume.

In [ ]:
def series_to_json_obj(ts):
    tsdictjson={'start':str(ts.index[0],'target':list(ts))}
    
    return tsdictjson

    pass

In [ ]:
# test out the code
series=timeseries[0]
tsjson=series_to_json_obj(ts)

### Saving Data, Locally

The next helper function will write one series to a single JSON line, using the new line character '\n'. The data is also encoded and written to a filename that we specify.

In [ ]:
# import json for formatting data
import json
import os

def write_json_dataset(timeseries, filename):
    for s in timeseries:
        jsonline=json.dumps(s)+'\n'
        jsonline=json:line.encode('utf-8')
        f.write(jsonline)

In [ ]:
# save this data to a local directory
dir = 'json_energy_data'
    
if not os.path.exists(dir):
    os.makedirs(dir)

In [ ]:
# directories to save train/test data

trainfile=os.path.join(dir, 'train.json')
testfile=os.path.join(dir,'test.json')

write_json_dataset(trainseries,trainfile)
write_json_dataset(timeseries,testfile)

---
## Uploading Data to S3

Next, to make this data accessible to an estimator, I'll upload it to S3.

### Sagemaker resources

Let's start by specifying:
* The sagemaker role and session for training a model.
* A default S3 bucket where we can save our training, test, and model data.

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [ ]:
# session, role, bucket
sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = sagemaker_session.default_bucket()


### EXERCISE: Upoad *both* training and test JSON files to S3

Specify *unique* train and test prefixes that define the location of that data in S3.
* Upload training data to a location in S3, and save that location to `train_path`
* Upload test data to a location in S3, and save that location to `test_path`

In [ ]:
# suggested that you set prefixes for directories in S3
trainprefix='{}/{}'.format(prefix, 'train')
testprefix='{}/{}'.format(prefix,'test''{}/{}'.format(prefix, 'train')
# upload data to S3, and save unique locations
trainpath = sagemaker_session.upload_data(trainfile, bucket=bucket, key_prefix=trainprefix)
testpath = sagemaker_session.upload_data(testfile, bucket=bucket,key_prefix=testprefix)

In [ ]:
# check locations
print('Training data is stored in: '+ trainpath)
print('Test data is stored in: '+ testpath)

---
# Training a DeepAR Estimator

Some estimators have specific, SageMaker constructors, but not all. Instead you can create a base `Estimator` and pass in the specific image (or container) that holds a specific model.

Next, we configure the container image to be used for the region that we are running in.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.amazon.amazon-estimator import et_image_uri
img=get_image_uri(boto3.Session().region_name
             'forecasting-deepar')

### EXERCISE: Instantiate an Estimator 

You can now define the estimator that will launch the training job. A generic Estimator will be defined by the usual constructor arguments and an `image_name`. 
> You can take a look at the [estimator source code](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/estimator.py#L595) to view specifics.


In [ ]:
from sagemaker.estimator import Estimator

# instantiate a DeepAR estimator
outdirs3="s3://{}/{}/output".format(bucket, prefix)
estimator = Estimator(sagemaker_session=sagemaker_session,
                      image_name=image_name,
                      role=role,
                      train_instance_count=1,
                      train_instance_type='ml.c4.xlarge',
                      output_path=outdirs3
                      )


## Setting Hyperparameters

Next, we need to define some DeepAR hyperparameters that define the model size and training behavior. Values for the epochs, frequency, prediction length, and context length are required.

* **epochs**: The maximum number of times to pass over the data when training.
* **time_freq**: The granularity of the time series in the dataset ('D' for daily).
* **prediction_length**: A string; the number of time steps (based off the unit of frequency) that the model is trained to predict. 
* **context_length**: The number of time points that the model gets to see *before* making a prediction. 

### Context Length

Typically, it is recommended that you start with a `context_length`=`prediction_length`. This is because a DeepAR model also receives "lagged" inputs from the target time series, which allow the model to capture long-term dependencies. For example, a daily time series can have yearly seasonality and DeepAR automatically includes a lag of one year. So, the context length can be shorter than a year, and the model will still be able to capture this seasonality. 

The lag values that the model picks depend on the frequency of the time series. For example, lag values for daily frequency are the previous week, 2 weeks, 3 weeks, 4 weeks, and year. You can read more about this in the [DeepAR "how it works" documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_how-it-works.html).

### Optional Hyperparameters

You can also configure optional hyperparameters to further tune your model. These include parameters like the number of layers in our RNN model, the number of cells per layer, the likelihood function, and the training options, such as batch size and learning rate. 

For an exhaustive list of all the different DeepAR hyperparameters you can refer to the DeepAR [hyperparameter documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html).

In [ ]:
frequency='D'

ctxtlenth=30

hyperparameters={
    "epochs": "100",
    "time_freq": frequency,
    "prediction_length": str(predictionlength),
    "context_length": str(ctxtlenth),
    "num_cells": "50",
    "num_layers": "2",
    "mini_batch_size": "128",
    "learning_rate": "0.05",
    "early_stopping_patience": "10"
}


In [ ]:
# set the hyperparams
estimator.set_hyperparameters(**hyperparameters)

## Training Job

Now, we are ready to launch the training job! SageMaker will start an EC2 instance, download the data from S3, start training the model and save the trained model.

If you provide the `test` data channel, as we do in this example, DeepAR will also calculate accuracy metrics for the trained model on this test data set. This is done by predicting the last `prediction_length` points of each time series in the test set and comparing this to the *actual* value of the time series. The computed error metrics will be included as part of the log output.

The next cell may take a few minutes to complete, depending on data size, model complexity, and training options.

In [ ]:
%%time

datapaths={
    'train':trainpath
    'test': testpath
}

estimator.fit(inputs=datapaths)

## Deploy and Create a Predictor

Now that we have trained a model, we can use it to perform predictions by deploying it to a predictor endpoint.

Remember to **delete the endpoint** at the end of this notebook. A cell at the very bottom of this notebook will be provided, but it is always good to keep, front-of-mind.

In [ ]:
%%time
#deployed model = predictor
deployeddeeparmodel=estimator.deploy(
    initial_instance_count=1, instance_type='ml.t2.medium', content_type="application/json")


---
# Generating Predictions

According to the [inference format](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar-in-formats.html) for DeepAR, the `predictor` expects to see input data in a JSON format, with the following keys:
* **instances**: A list of JSON-formatted time series that should be forecast by the model.
* **configuration** (optional): A dictionary of configuration information for the type of response desired by the request.

Within configuration the following keys can be configured:
* **num_samples**: An integer specifying the number of samples that the model generates when making a probabilistic prediction.
* **output_types**: A list specifying the type of response. We'll ask for **quantiles**, which look at the list of num_samples generated by the model, and generate [quantile estimates](https://en.wikipedia.org/wiki/Quantile) for each time point based on these values.
* **quantiles**: A list that specified which quantiles estimates are generated and returned in the response.


Below is an example of what a JSON query to a DeepAR model endpoint might look like.

```
{
 "instances": [
  { "start": "2009-11-01 00:00:00", "target": [4.0, 10.0, 50.0, 100.0, 113.0] },
  { "start": "1999-01-30", "target": [2.0, 1.0] }
 ],
 "configuration": {
  "num_samples": 50,
  "output_types": ["quantiles"],
  "quantiles": ["0.5", "0.9"]
 }
}
```


## JSON Prediction Request

The code below accepts a **list** of time series as input and some configuration parameters. It then formats that series into a JSON instance and converts the input into an appropriately formatted JSON_input.

In [ ]:
def json_predictor_input(inputseries, nrsamples=50, quantiles=['0.1', '0.5', '0.9']):
    
    helperjsonlist=[]
    for pt in range(len(inputseries)):
        helperjsonlist.append(inputseries[pt])
        
    outputconfiguration={"num_samples": num_samples, 
                     "output_types": ["quantiles"], 
                     "quantiles": quantiles}
    outputinfo={"instances": instances, 
                    "configuration": configuration}
    jsonoutput=json.dumps(outputinfo).encode('utf-8')
    
    return jsonoutput                     

### Get a Prediction

We can then use this function to get a prediction for a formatted time series!

In the next cell, I'm getting an input time series and known target, and passing the formatted input into the predictor endpoint to get a resultant prediction.

In [ ]:
# target: An array of numerical values that represent the time series.
targetseries=timeseries

modelinputjson=json_predictor_input(trainseries) #function defined above
predjson=deployeddeeparmodel.predict(modelinputjson)predjson

print(predjson)

## Decoding Predictions

The predictor returns JSON-formatted prediction, and so we need to extract the predictions and quantile data that we want for visualizing the result. The function below, reads in a JSON-formatted prediction and produces a list of predictions in each quantile.

In [ ]:
# helper function to decode JSON prediction
def decode_prediction(prediction, encoding='utf-8'):
    #Accepts a JSON prediction and returns a list of prediction data.
    prediction_data = json.loads(prediction.decode(encoding))
    prediction_list = []
    for k in range(len(prediction_data['predictions'])):
        prediction_list.append(pd.DataFrame(data=prediction_data['predictions'][k]['quantiles']))
    return prediction_list

def decode_prediction(predictioninput, encoding='utf-8'):
    predicted=json.loads(predictioninput.decode(encoding))
    helperpredlst=[]
    for idx in range(length(predicted['predictions'])):
        helperpredlst.append(pd.DataFrame(data=predicted['predictions'][idx]['quantiles']))
        
        return helperpredlst


In [ ]:
predictionlist = decode_prediction(predjson)
print(predictionlist[0])

## Display the Results!

The quantile data will give us all we need to see the results of our prediction.
* Quantiles 0.1 and 0.9 represent higher and lower bounds for the predicted values.
* Quantile 0.5 represents the median of all sample predictions.


In [ ]:
# display the prediction median against the actual data
def display_quantiles(len(predictionlist, targetseries)):
    for idx in range(len(predictionlist)):
        plt.figure(figsize=(12,6))
        if targetseries is not None:
            target=targetseries[idx][-predictionlength:]
            plt.plot(range(len(target)), target, label='target' color='k')
            p10=predictionlist[idx]['0.1']
            p90=predictionlist[idx]['0.9']
            plt.plot(range(len(target)), p10, label='p10' color='k')
            plt.plot(range(len(target)), p90, label='p90' color='k')
            predictionlist[idx]['0.5'.plot(label='prediction median', color='k']
            plt.show()

In [ ]:
# display predictions
display_quantiles(predictionlist, targetseries)

## Predicting the Future

Recall that we did not give our model any data about 2010, but let's see if it can predict the energy consumption given **no target**, only a known start date!

### EXERCISE: Format a request for a "future" prediction

Create a formatted input to send to the deployed `predictor` passing in my usual parameters for "configuration". The "instances" will, in this case, just be one instance, defined by the following:
* **start**: The start time will be time stamp that you specify. To predict the first 30 days of 2010, start on Jan. 1st, '2010-01-01'.
* **target**: The target will be an empty list because this year has no, complete associated time series; we specifically withheld that information from our model, for testing purposes.
```
{"start": start_time, "target": []} # empty target
```

In [ ]:
startdate = '2010-01-01'
timept = '00:00:00'
starttime = startdate +' '+ timept

requestdata={'instances':[{'start':starttime,'target':[]}],
             'configuration': {'nrsamples': 50,
                               'output_types':['quantiles'],
                               'quantiles': ['o.q','0.5', '0.9']}
            }

predinputjson=json.dumps(requestdata).encode('utf-8')

Then get and decode the prediction response, as usual.

In [ ]:
# get prediction response
predjson=deployeddeeparmodel.predict(predinputjson)
pred2010=decode_prediction(predjson)

Finally, I'll compare the predictions to a known target sequence. This target will come from a time series for the 2010 data, which I'm creating below.

In [ ]:
tseries2010=[]
df2010=meanpowerdf.values[1112:]
idx=pd.DatetimeIndex(start=startdate, periods=len(df2010),freq='D')
tseries2010.append(pd.Series(data=df2010,index=idx))

In [ ]:
startindex=0
endindex=startindex+prediction_length # hyperparameter
targetseries2010=[tseries2010[0][startindex:endindex]]
display_quantiles(pred2010,targetseries2010)

## Delete the Endpoint

Try your code out on different time series. You may want to tweak your DeepAR hyperparameters and see if you can improve the performance of this predictor.

When you're done with evaluating the predictor (any predictor), make sure to delete the endpoint.

In [ ]:
## TODO: delete the endpoint
deployeddeeparmodel.delete_endpoint()

## Conclusion

Now you've seen one complex but far-reaching method for time series forecasting. You should have the skills you need to apply the DeepAR model to data that interests you!